In [3]:
import findspark
findspark.init()
findspark.find()

'C:\\Spark'

In [4]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [5]:
import pyspark
from pyspark import SparkContext

In [6]:

import datetime
import datetime
from pyspark.sql.functions import year, month, dayofmonth, expr,col
from pyspark.sql.types import DateType, IntegerType

In [7]:
#import pyspark
from pyspark.sql import SparkSession #Importing the Libraries
# Creating Spark Session
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

In [8]:
df_branch = spark.read.format('org.apache.spark.sql.json').load("cdw_sapp_branch.json")
df_branch.show()

+-----------------+-----------+------------+------------+------------+-------------------+----------+--------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|      BRANCH_STREET|BRANCH_ZIP|        LAST_UPDATED|
+-----------------+-----------+------------+------------+------------+-------------------+----------+--------------------+
|        Lakeville|          1|Example Bank|  1234565276|          MN|       Bridle Court|     55044|2018-04-18T16:51:...|
|          Huntley|          2|Example Bank|  1234618993|          IL|  Washington Street|     60142|2018-04-18T16:51:...|
|SouthRichmondHill|          3|Example Bank|  1234985926|          NY|      Warren Street|     11419|2018-04-18T16:51:...|
|       Middleburg|          4|Example Bank|  1234663064|          FL|   Cleveland Street|     32068|2018-04-18T16:51:...|
|    KingOfPrussia|          5|Example Bank|  1234849701|          PA|        14th Street|     19406|2018-04-18T16:51:...|
|         Paters

In [9]:
df_branch.printSchema()

root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)



In [10]:
df_branch = df_branch.withColumn("LAST_UPDATED",col("LAST_UPDATED").cast("timestamp"))
df_branch = df_branch.fillna( {'BRANCH_ZIP': 00000} )

In [47]:
from pyspark.sql.functions import regexp_replace

df_branch = df_branch.withColumn('BRANCH_PHONE', regexp_replace('BRANCH_PHONE', r'(^\d{3})(\d{3})(\d{4})', r'($1)$2-$3'))
df_branch.show(2)



+-----------+-----------+------------+-------------+------------+-----------------+----------+-------------------+-------------+
|BRANCH_CITY|BRANCH_CODE| BRANCH_NAME| BRANCH_PHONE|BRANCH_STATE|    BRANCH_STREET|BRANCH_ZIP|       LAST_UPDATED|     replaced|
+-----------+-----------+------------+-------------+------------+-----------------+----------+-------------------+-------------+
|  Lakeville|          1|Example Bank|(123)456-5276|          MN|     Bridle Court|     55044|2018-04-18 15:51:47|(123)456-5276|
|    Huntley|          2|Example Bank|(123)461-8993|          IL|Washington Street|     60142|2018-04-18 15:51:47|(123)461-8993|
+-----------+-----------+------------+-------------+------------+-----------------+----------+-------------------+-------------+
only showing top 2 rows



In [12]:
import pyspark.sql.functions as F
#df3 = df_branch.withColumn("BRANCH_PHONE", F.regexp_replace(F.col("BRANCH_PHONE"), "(\\d{3})(\\d{7})(\\d+)", "$1-$2-$3")) 
#df2 = df_branch.withColumn('BRANCH_PHONE', F.expr(concat_ws('-', substring(BRANCH_PHONE, 1, 3),substring(BRANCH_PHONE, 4, 6), substring(BRANCH_PHONE, 9, length(BRANCH_PHONE))
          
#df2.show(2)           

In [43]:
df_branch.printSchema()

root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = false)
 |-- LAST_UPDATED: timestamp (nullable = true)
 |-- replaced: string (nullable = true)



In [44]:
df_branch1 = df_branch.select("BRANCH_CODE","BRANCH_NAME","BRANCH_STREET","BRANCH_CITY","BRANCH_STATE","BRANCH_ZIP","BRANCH_PHONE","LAST_UPDATED")
df_branch1.show(2)

+-----------+------------+-----------------+-----------+------------+----------+-------------+-------------------+
|BRANCH_CODE| BRANCH_NAME|    BRANCH_STREET|BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP| BRANCH_PHONE|       LAST_UPDATED|
+-----------+------------+-----------------+-----------+------------+----------+-------------+-------------------+
|          1|Example Bank|     Bridle Court|  Lakeville|          MN|     55044|(123)456-5276|2018-04-18 15:51:47|
|          2|Example Bank|Washington Street|    Huntley|          IL|     60142|(123)461-8993|2018-04-18 15:51:47|
+-----------+------------+-----------------+-----------+------------+----------+-------------+-------------------+
only showing top 2 rows



In [46]:
df_branch1.write.format("jdbc")\
.mode("append")\
.option("driver","com.mysql.cj.jdbc.Driver")\
.option("user","root")\
.option("url","jdbc:mysql://localhost:3307/creditcard_capstone")\
.option("dbtable","creditcard_capstone.CDW_SAPP_BRANCH")\
.option("password","")\
.save()